# Ordinary Degreedays between April and October

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import datetime as dt
import os
import re

In [ ]:
# All directories
root       = "C:/Users/freiste/OneDrive - Ilmatieteen laitos/Documents/IIASA YSSP 2023"
this_dir   =  os.getcwd()

input_dir  = f"{root}/02 - Data/EU/Copernicus_E-OBS_Weather_Original" # to save space, not all data might be there anymore
output_dir = f"{root}/02 - Data/EU/Copernicus_E-OBS_Weather_Postprocessed"

### Methods

In [ ]:
def degreedays(yr):
    
    if 1980 <= yr <= 1994:
        strtyr, endyr = 1980, 1994
    elif 1995 <= yr <= 2010:
        strtyr, endyr = 1995, 2010
    elif 2011 <= yr <= 2022:
        strtyr, endyr = 2011, 2022
    else:
        print('Error, year unknown.')
    
    # Open Data
    t = xr.open_dataset(f"{obs_path}/tempavg_dailymean_{strtyr}-{endyr}_0.25deg.nc")
    t.close()
    
    
    
    # Filter out bark beetle season
    idx1 = t.indexes["time"].get_loc(f"{yr}-04-01T00:00:00",  method="nearest")
    idx2 = t.indexes["time"].get_loc(f"{yr}-10-31T00:00:00",  method="nearest")
    
    t = t.tg.isel(time=slice(idx1, idx2+1))
    t = t.sum('time')
    
    
    t = t.assign_coords({'year' : yr})
    t = t.where(t > 0)
    
    return t.to_dataset(name='degreedays')


### Execution

In [ ]:
yrs = range(1980,2023)

for y, yr in enumerate(yrs):
    
    if y==0:
        cumdegreedays = degreedays(yr)
    else:
        cumdegreedays = xr.concat([cumdegreedays, degreedays(yr)], dim="year")

        
cumdegreedays = \
cumdegreedays.assign_attrs({'cumulative degree days' : 'Cumulative sum of daily average temperature between April 1st and October 31st'})

# Export 
cumdegreedays.to_netcdf(f"{obs_path}/cumulative_degreedays_europe_1980-2022_0.25deg.nc")
cumdegreedays